# <ins>Sentiment Analysis: "Covid Vaccine" on Twitter</ins>
## Data Cleaning

In this notebook, I pick up with the data I scraped from Twitter using [Twint](https://github.com/twintproject/twint) in my [web_scraping.ipynb](https://github.com/davidlebruce/covid_vaccine_tweets/blob/main/01_web_scraping.ipynb) notebook. The only packages I used for this step of data cleaning were Pandas and datetime. I used Pandas to identify the columns with mostly missing values and those which might be helpful for modeling, as well as the rows which contained tweets in languages other than English and any duplicates. With datetime I was able to convert the date column into a datetime object and drop the timestamp. The process and code are outlined below. Should you want to skip ahead to the exploratory data analysis, you can follow the link to the next notebook, [EDA](https://github.com/davidlebruce/covid_vaccine_tweets/blob/main/03_EDA.ipynb).

## Read in CSV and Look at Data

In [1]:
import pandas as pd
import datetime

In [2]:
# read in data
df = pd.read_csv('./data/covid_vaccine_tweets.csv')

In [3]:
df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1233542386669641728,1233542386669641728,1.582934e+12,2020-02-28 18:59:57,-500,NaN,"For everyone comparing COVID-19 to the flu, ju...",en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1,1233542196193824769,1233542196193824769,1.582934e+12,2020-02-28 18:59:12,-500,NaN,"“Perhaps more than any other type of medicine,...",en,"['vaccine', 'covid_19', 'coronavirus']",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,2,1233542052161507328,1233542052161507328,1.582934e+12,2020-02-28 18:58:37,-500,NaN,Provide pharmaceutical companies with the oppo...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,3,1233541922108694528,1233541922108694528,1.582934e+12,2020-02-28 18:58:06,-500,NaN,From John's Hopkins: 1) Immediate risk of COV...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,4,1233541049903177728,1233541049903177728,1.582934e+12,2020-02-28 18:54:39,-500,NaN,If the #stockmarkets keep falling - how confid...,en,"['stockmarkets', 'vaccine', 'covid']",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [4]:
# shape to begin with
df.shape

(80730, 39)

In [5]:
# original columns
df.columns

Index(['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'date', 'timezone',
       'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id',
       'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls',
       'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

## Check for NaNs and Abnormalities

In [6]:
# check columns for null values
df.isna().sum()

Unnamed: 0             0
id                     0
conversation_id        0
created_at             0
date                   0
timezone               0
place              80699
tweet                  0
language               0
hashtags               0
cashtags               0
user_id                0
user_id_str            0
username               0
name                   3
day                    0
hour                   0
link                   0
urls                   0
photos                 0
video                  0
thumbnail          71282
retweet                0
nlikes                 0
nreplies               0
nretweets              0
quote_url          75067
search                 0
near               80730
geo                80730
source             80730
user_rt_id         80730
user_rt            80730
retweet_id         80730
reply_to               0
retweet_date       80730
translate          80730
trans_src          80730
trans_dest         80730
dtype: int64

In [7]:
# not all tweets are in English as the scraper had specified
df.language.value_counts()[:5]

en     78136
und      503
tl       426
ja       214
fr       203
Name: language, dtype: int64

In [8]:
# remove non-English tweets
df = df[df['language'] == 'en']

In [9]:
# all tweets appear to be scraped from Eastern timezone
df.timezone.value_counts()

-500    78136
Name: timezone, dtype: int64

In [10]:
# all False values for 'retweet'
df.retweet.value_counts()

False    78136
Name: retweet, dtype: int64

## Drop Unnecessary Columns

In [11]:
# drop columns full of null values, and 'Unnamed: 0'
df = df.drop(labels=['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'timezone',
       'place', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'name',
       'day', 'hour', 'urls', 'photos', 'video', 'thumbnail', 'retweet', 'quote_url', 
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 
       'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest'], axis=1)

In [12]:
# remaining column names
df.columns

Index(['date', 'tweet', 'username', 'link', 'nlikes', 'nreplies', 'nretweets'], dtype='object')

In [13]:
# check the shape
df.shape

(78136, 7)

## Check and Update Datatypes

In [14]:
# checking data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78136 entries, 0 to 80729
Data columns (total 7 columns):
date         78136 non-null object
tweet        78136 non-null object
username     78136 non-null object
link         78136 non-null object
nlikes       78136 non-null int64
nreplies     78136 non-null int64
nretweets    78136 non-null int64
dtypes: int64(3), object(4)
memory usage: 4.8+ MB


In [15]:
# convert date to datetime object
df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# convert datetime to string without timestamp
df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

# convert date back to datetime
df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

In [16]:
# check the current edition
df.head()

,date,tweet,username,link,nlikes,nreplies,nretweets
0,2020-02-28,"For everyone comparing COVID-19 to the flu, ju...",MonaIbrahim,https://twitter.com/MonaIbrahim/status/1233542...,3,0,0
1,2020-02-28,"“Perhaps more than any other type of medicine,...",Moratel_Entles,https://twitter.com/Moratel_Entles/status/1233...,1,0,0
2,2020-02-28,Provide pharmaceutical companies with the oppo...,help_4_Iranian,https://twitter.com/help_4_Iranian/status/1233...,0,0,0
3,2020-02-28,From John's Hopkins: 1) Immediate risk of COV...,TruthFreedom17,https://twitter.com/TruthFreedom17/status/1233...,1,1,0
4,2020-02-28,If the #stockmarkets keep falling - how confid...,notarywise,https://twitter.com/notarywise/status/12335410...,0,0,0


## Drop Duplicate Rows

In [17]:
# checking for duplicate tweets
df.duplicated(subset='tweet').value_counts()

False    77259
True       877
dtype: int64

In [18]:
# dropping duplicates
df = df.drop_duplicates(subset='tweet')

## Save as a Pickle

In [19]:
# save as a pickle to pickle_jar and final shape
df.to_pickle('data/clean_df.pkl')

# final shape
print(df.shape)

(77259, 7)


The final shape of my dataframe after the cleaning process is 77,259 tweets and 7 columns of data per tweet. It is unfortunate to have had to let go of almost 3,000 rows of data, but the decisions to drop non-English and duplicate tweets are justified as those data points would only cloud my analysis and modeling. Over a span of 10 months, this many tweets is substantial for general analysis of Enlish language tweets about the COVID-19 vaccine. I will begin visualizing and preprocessing the data in my next notebook titled [EDA](https://github.com/davidlebruce/covid_vaccine_tweets/blob/main/03_EDA.ipynb).